In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
df.head()

In [3]:
df.shape

In [4]:
df.sample(5)

In [5]:
import matplotlib.pyplot as plt

In [6]:
plt.imshow(df.iloc[37118,1:].values.reshape(28,28))

In [7]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [43]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test  = train_test_split(X,y,test_size=0.2, random_state=41)

In [9]:
from sklearn.neighbors import KNeighborsClassifier

In [10]:
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)

In [14]:
import time
start = time.time()
y_pred = knn.predict(X_test)
print(time.time() - start)

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

#  Appling PCA and Founding Optimal value for model features

In [44]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [56]:
X_train_st = scaler.fit_transform(X_train)
X_test_st = scaler.transform(X_test)

In [54]:
# Appling PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=200)

In [57]:
X_train_trf = pca.fit_transform(X_train_st)
X_test_trf = pca.transform(X_test_st)

In [48]:
X_test_trf.shape

In [34]:
knn = KNeighborsClassifier()

In [58]:
knn.fit(X_train_trf,y_train)

In [59]:
start = time.time()
y_pred = knn.predict(X_test_trf)
print(time.time()-start)

In [60]:
accuracy_score(y_pred,y_test)

* Accuracy Reduce due to less no. of feature selection

In [62]:
start = time.time()
print("_____Process____Started______")
for i in range(200,450,10):
    pca = PCA(n_components=200)
    # trandforming data
    X_train_trf = pca.fit_transform(X_train_st)
    X_test_trf = pca.transform(X_test_st)
    
    knn = KNeighborsClassifier()
    
    knn.fit(X_train_trf,y_train)
    
    y_pred = knn.predict(X_test_trf)
    
    print(i, " : ", accuracy_score(y_pred,y_test))

print("Total Time Taken : ", start - time.time()," s")

1. **Transforming Data For 3D visulisation**

In [64]:
#PCA for 3 feature
pca = PCA(n_components=3)
X_train_trf = pca.fit_transform(X_train_st)
X_test_trf = pca.transform(X_test_st)

In [66]:
X_train_trf.shape

In [70]:
import plotly.express as px
fig = px.scatter_3d(x = X_train_trf[:,0],
                    y = X_train_trf[:,1],
                    z = X_train_trf[:,2],
                    color = y_train)
fig.update_layout(margin = dict(l=20 , r=20 , t=20, b=20))
fig.show()

In [72]:
#Eigen value
pca.explained_variance_

In [74]:
#Eigen Vectors
pca.components_.shape
# we hav 3 eigen values for 784 rows 

In [77]:
# With this we can calculate the importance of each Features
pca.explained_variance_ratio_*100

## Finding Optimal No. of Feature using Eigen Values


In [79]:
pca = PCA(n_components=None)
X_trf  = pca.fit_transform(X)

In [83]:
#no. of eifen vectors:
pca.components_.shape
# we have 784 unit vector and each vactor have 784 eigon vector of 784 Featurs

In [84]:
# With this we can calculate the importance of each Features
e_val= pca.explained_variance_ratio_*100

In [85]:
#We need to select up to 90% of values
np.cumsum(e_val)

In [86]:
#plotting cumsum value for better visulistion
plt.figure(figsize = (10,8))
plt.plot(np.cumsum(e_val))
plt.show()

1. **feature between 150-250 will work better for 90% of classification**